# 使用LLaMA Factory微调Llama3模型

[LLaMA Factory](https://github.com/hiyouga/LLaMA-Factory)是一款开源低代码大模型微调框架，集成了业界最广泛使用的微调技术，支持通过Web UI界面零代码微调大模型，目前已经成为开源社区内最受欢迎的微调框架，GitHub星标超过2万。本教程将基于Meta AI开源的Llama-3 8B模型，介绍如何使用PAI平台及LLaMA Factory训练框架完成模型的中文化与角色扮演微调和评估。 

## 运行环境要求

- GPU推荐使用24GB显存的A10（`ecs.gn7i-c8g1.2xlarge`）或更高配置
- 镜像选择DSW官方镜像`modelscope:1.18.0-pytorch2.3.0-gpu-py310-cu121-ubuntu22.04`

## 系统环境检查

In [1]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                              |
|:-------------|:----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                          |
| CPU 信息     | Intel(R) Xeon(R) Platinum 8369B CPU @ 2.70GHz (4 physical cores, 8 logical cores) |
| 内存信息     | 28.91 GB (Available: 27.44 GB)                                                    |
| GPU 信息     | NVIDIA A10 (23028 MiB)                                                            |
| CUDA 信息    | 12.1                                                                              |
| Python 版本  | 3.10.14                                           

## 检查软硬件依赖
| 必需项        | 至少     | 推荐      |
| ------------ | ------- | --------- |
| python       | 3.9     | 3.10      |
| torch        | 2.0.0   | 2.6.0     |
| torchvision  | 0.15.0  | 0.21.0    |
| transformers | 4.49.0  | 4.50.0    |
| datasets     | 2.16.0  | 3.2.0     |
| accelerate   | 0.34.0  | 1.2.1     |
| peft         | 0.14.0  | 0.15.1    |
| trl          | 0.8.6   | 0.9.6     |

| 可选项        | 至少     | 推荐      |
| ------------ | ------- | --------- |
| CUDA         | 11.6    | 12.2      |
| deepspeed    | 0.10.0  | 0.16.4    |
| bitsandbytes | 0.39.0  | 0.43.1    |
| vllm         | 0.4.3   | 0.8.2     |
| flash-attn   | 2.5.6   | 2.7.2     |

### 硬件依赖

\* *估算值*

| 方法                             | 精度 |   7B  |  14B  |  30B  |   70B  |   `x`B  |
| ------------------------------- | ---- | ----- | ----- | ----- | ------ | ------- |
| Full (`bf16` or `fp16`)         |  32  | 120GB | 240GB | 600GB | 1200GB | `18x`GB |
| Full (`pure_bf16`)              |  16  |  60GB | 120GB | 300GB |  600GB |  `8x`GB |
| Freeze/LoRA/GaLore/APOLLO/BAdam |  16  |  16GB |  32GB |  64GB |  160GB |  `2x`GB |
| QLoRA                           |   8  |  10GB |  20GB |  40GB |   80GB |   `x`GB |
| QLoRA                           |   4  |   6GB |  12GB |  24GB |   48GB | `x/2`GB |
| QLoRA                           |   2  |   4GB |   8GB |  16GB |   24GB | `x/4`GB |

In [2]:
import sys
import platform
import importlib.metadata
from packaging import version
import torch

# ================= 您的最新配置要求 =================
REQUIRED_PACKAGES = {
    "python":       ("3.9",    "3.10"),
    "torch":        ("2.0.0",  "2.6.0"),
    "torchvision":  ("0.15.0", "0.21.0"),
    "transformers": ("4.49.0", "4.50.0"),
    "datasets":     ("2.16.0", "3.2.0"),
    "accelerate":   ("0.34.0", "1.2.1"),
    "peft":         ("0.14.0", "0.15.1"),
    "trl":          ("0.8.6",  "0.9.6"),
}

OPTIONAL_PACKAGES = {
    "cuda":         ("11.6",   "12.2"),
    "deepspeed":    ("0.10.0", "0.16.4"),
    "bitsandbytes": ("0.39.0", "0.43.1"),
    "vllm":         ("0.4.3",  "0.8.2"),
    "flash-attn":   ("2.5.6",  "2.7.2"),
}

# ================= 检测逻辑 =================
def get_package_version(package_name):
    try:
        if package_name == "python": return platform.python_version()
        elif package_name == "cuda": return torch.version.cuda if torch.cuda.is_available() else None
        elif package_name == "flash-attn": return importlib.metadata.version("flash_attn")
        else: return importlib.metadata.version(package_name)
    except importlib.metadata.PackageNotFoundError: return None

def check_requirement(name, constraints, is_optional=False):
    min_v, rec_v = constraints
    current_v = get_package_version(name)
    
    if current_v is None:
        status, msg, color = ("⚪", "未安装 (可选)", "\033[90m") if is_optional else ("❌", "未安装 !!", "\033[91m")
    else:
        try:
            curr_p, min_p, rec_p = version.parse(current_v), version.parse(min_v), version.parse(rec_v)
            if curr_p >= rec_p:   status, msg, color = ("✅", f"完美 ({current_v})", "\033[92m")
            elif curr_p >= min_p: status, msg, color = ("⚠️", f"达标 ({current_v})", "\033[93m")
            else:                 status, msg, color = ("❌", f"过低 ({current_v})", "\033[91m")
        except: status, msg, color = ("❓", f"解析错误", "\033[93m")

    print(f"{color}{status} {name:<15} | 当前: {msg:<15} | 必需: >={min_v:<8} | 推荐: >={rec_v}\033[0m")

print("="*65)
print(f"🛠️  环境依赖检测报告 (基于您提供的最新标准)")
print("="*65)
print("[ 必需项 ]")
for k, v in REQUIRED_PACKAGES.items(): check_requirement(k, v)
print("\n[ 可选项 ]")
for k, v in OPTIONAL_PACKAGES.items(): check_requirement(k, v, True)
print("="*65)

🛠️  环境依赖检测报告 (基于您提供的最新标准)
[ 必需项 ]
✅ python          | 当前: 完美 (3.10.14)    | 必需: >=3.9      | 推荐: >=3.10
⚠️ torch           | 当前: 达标 (2.3.1)      | 必需: >=2.0.0    | 推荐: >=2.6.0
⚠️ torchvision     | 当前: 达标 (0.18.1)     | 必需: >=0.15.0   | 推荐: >=0.21.0
✅ transformers    | 当前: 完美 (4.57.1)     | 必需: >=4.49.0   | 推荐: >=4.50.0
⚠️ datasets        | 当前: 达标 (2.20.0)     | 必需: >=2.16.0   | 推荐: >=3.2.0
✅ accelerate      | 当前: 完美 (1.11.0)     | 必需: >=0.34.0   | 推荐: >=1.2.1
✅ peft            | 当前: 完美 (0.17.1)     | 必需: >=0.14.0   | 推荐: >=0.15.1
✅ trl             | 当前: 完美 (0.9.6)      | 必需: >=0.8.6    | 推荐: >=0.9.6

[ 可选项 ]
⚠️ cuda            | 当前: 达标 (12.1)       | 必需: >=11.6     | 推荐: >=12.2
⚠️ deepspeed       | 当前: 达标 (0.14.4)     | 必需: >=0.10.0   | 推荐: >=0.16.4
✅ bitsandbytes    | 当前: 完美 (0.43.2)     | 必需: >=0.39.0   | 推荐: >=0.43.1
⚪ vllm            | 当前: 未安装 (可选)        | 必需: >=0.4.3    | 推荐: >=0.8.2
⚠️ flash-attn      | 当前: 达标 (2.5.9.post1) | 必需: >=2.5.6    | 推荐: >=2.7.2


## 1. 安装LLaMA Factory


首先，拉取LLaMA-Factory项目到DSW实例。

In [15]:
# 定义仓库文件夹名称
repo_name = "LLaMA-Factory"

# 1. 判断文件夹是否存在
if os.path.exists(repo_name):
    print(f"⚠️ 检测到旧目录 '{repo_name}'，正在删除...")
    # 强制删除 (相当于 rm -rf)
    shutil.rmtree(repo_name) 
    print("🗑️ 旧目录已清除！")
else:
    print(f"✅ 目录 '{repo_name}' 不存在，准备新建...")

✅ 目录 'LLaMA-Factory' 不存在，准备新建...


In [16]:
# 1. 克隆仓库（使用国内镜像）
!git clone -b v0.9.2 https://github.com/hiyouga/LLaMA-Factory.git

正克隆到 'LLaMA-Factory'...
remote: Enumerating objects: 24474, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 24474 (delta 17), reused 8 (delta 8), pack-reused 24438 (from 3)
接收对象中: 100% (24474/24474), 12.15 MiB | 1.17 MiB/s, 完成.
处理 delta 中: 100% (17639/17639), 完成.
注意：正在切换到 'e2299e261be852304bb1d370515078193ab12bd8'。

您正处于分离头指针状态。您可以查看、做试验性的修改及提交，并且您可以在切换
回一个分支时，丢弃在此状态下所做的提交而不对分支造成影响。

如果您想要通过创建分支来保留在此状态下所做的提交，您可以通过在 switch 命令
中添加参数 -c 来实现（现在或稍后）。例如：

  git switch -c <新分支名>

或者撤销此操作：

  git switch -

通过将配置变量 advice.detachedHead 设置为 false 来关闭此建议



接着，我们安装LLaMA-Factory依赖环境。

In [19]:
# 1. 【关键】使用 %cd 进入目录 (不要用 !cd)
%cd /mnt/workspace/demos/llama_factory/LLaMA-Factory

# 2. 去掉无关依赖
!pip install --upgrade pip
!pip uninstall -y pai-easycv
!pip uninstall -y autoawq
!pip uninstall -y ms-swift
!pip uninstall -y lmdeploy

# 3. 安装项目依赖
# 这会自动安装 v0.9.2 所需的其他库
!pip install -e ".[torch,metrics]"

[Errno 2] No such file or directory: 'LLaMA-Factory'
/mnt/workspace/demos/llama_factory/LLaMA-Factory/LLaMA-Factory
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
    torch (>=1.9.*)
           ~~~~~~^
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Obtaining file:///mnt/workspace/demos/llama_factory/LLaMA-Factory/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.2-0.editable-py3-none-any.whl size=25820 sha256=ea4c02997d779dfa72602f8a261f75bd31c69f3d7634c3b6f89650bce129c406
  Stored in directory: /tmp/pip-ephem-wheel-cache-y2iw4es5/wheels/9f/39/78/cf31139d4a5de9f6586e31fdc7f2a6240817e7c6a901a723ff
Successfully built llamafactory
    torch (>=1.9.*)
           ~~~~~~^
  At

运行如下命令，如果显示llamafactory-cli的版本，则表示安装成功。

In [20]:
!llamafactory-cli version

[2025-11-25 15:40:06,420] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.2                |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------


## 2. 下载数据集

LLaMA-Factory项目内置了丰富的数据集，放在了`data`目录下。您可以跳过本步骤，直接使用内置数据集。您也可以准备自定义数据集，将数据处理为框架特定的格式，放在`data`下，并且修改`dataset_info.json`文件。

本教程准备了一份多轮对话数据集，运行下述命令下载数据。

In [ ]:
%cd LLaMA-Factory
!wget https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/llama_factory/data.zip
!mv data rawdata && unzip data.zip -d data

数据集中的样本为多轮对话形式，每轮对话都由一条用户指令和一条模型回答组成，微调过程中模型不断学习样本中的回答风格，从而达到中文化与角色扮演的目的。该数据集分为训练集和验证集两个文件，其中训练集共包含320组多轮对话样本，验证集共包含30组多轮对话样本，数据样例如下所示：

In [ ]:
[
{
    "conversations": [
      {
        "from": "human",
        "value": "模仿西游记的孙悟空的语气和我聊天。如果你把水冻住，你会得到什么？"
      },
      {
        "from": "gpt",
        "value": "哈哈，这问题还难倒俺老孙吗？把水冻住就是变成冰啦！不过若是在取经的路上用得着这冰来做啥，比如过火焰山时，那可就是解渴消暑的好东西了！你觉得呢？还有没有更有趣的问题让俺老孙来解答？"
      },
      {
        "from": "human",
        "value": "那怎么做才可以把水冻住"
      },
      {
        "from": "gpt",
        "value": "哈哈，对俺老孙来说，这可是轻而易举的事情！只需将水放在凉快的地方，或者使用法力，一吹仙气，水便能迅速结冰啦！当然，在你的世界里，把水放进冰箱的冷冻室里等一阵子，水就会变成冰块了。要是嫌慢，可以尝试用点干冰或液氮什么的，不过得小心使用哈！毕竟不像俺老孙有抗寒法力。"
      }
    ]
}
]

## 3. 模型微调
### 3.1 启动Web UI

做好前序准备工作后，直接运行下述命令就可以启动Web UI。这里用到的环境变量解释如下：
- `USE_MODELSCOPE_HUB`设为1，表示模型来源是ModelScope。使用HuggingFace模型可能会有网络问题。

点击返回的URL地址`http://0.0.0.0:7860`，进入Web UI页面。

In [ ]:
!export USE_MODELSCOPE_HUB=1 && \
llamafactory-cli webui

[2025-11-25 15:40:26,710] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


### 3.2 配置参数
进入WebUI后，可以切换到中文（zh）。首先配置模型，本教程选择LLaMA3-8B-Chat模型，微调方法则保持默认值lora，使用LoRA轻量化微调方法能极大程度地节约显存。
![image.png](_html/finetune_model.jpg)

数据集使用上述下载的`train.json`。
![image.png](_html/finetune_data.jpg)

可以点击「预览数据集」。点击关闭返回训练界面。
![image.png](_html/finetune_preview.jpg)

设置学习率为1e-4，梯度累积为2，有利于模型拟合。如果显卡是V100，计算类型保持为fp16；如果使用了A10，可以更改计算类型为bf16。
![image.png](_html/finetune_params.jpg)

点击LoRA参数设置展开参数列表，设置LoRA+学习率比例为16，LoRA+被证明是比LoRA学习效果更好的算法。在LoRA作用模块中填写all，即将LoRA层挂载到模型的所有线性层上，提高拟合效果。
![image.png](_html/finetune_lora.jpg)


### 3.3 启动微调
将输出目录修改为`train_llama3`，训练后的LoRA权重将会保存在此目录中。点击「预览命令」可展示所有已配置的参数，您如果想通过代码运行微调，可以复制这段命令，在命令行运行。

点击「开始」启动模型微调。
![image.png](_html/finetune_start.jpg)

启动微调后需要等待一段时间，待模型下载完毕后可在界面观察到训练进度和损失曲线。模型微调大约需要20分钟，显示“训练完毕”代表微调成功。
![image.png](_html/finetune_result.jpg)

## 4. 模型评估


微调完成后，点击页面顶部的「刷新适配器」，然后点击适配器路径，即可弹出刚刚训练完成的LoRA权重，点击选择下拉列表中的train_llama3选项，在模型启动时即可加载微调结果。
![image.png](_html/evaluate_adaptor.jpg)

选择「Evaluate&Predict」栏，在数据集下拉列表中选择「eval」（验证集）评估模型。更改输出目录为`eval_llama3`，模型评估结果将会保存在该目录中。最后点击开始按钮启动模型评估。
![image.png](_html/evaluate_start.jpg)

模型评估大约需要5分钟左右，评估完成后会在界面上显示验证集的分数。其中ROUGE分数衡量了模型输出答案（predict）和验证集中标准答案（label）的相似度，ROUGE分数越高代表模型学习得更好。
![image.png](_html/evaluate_result.jpg)


## 5. 模型对话

选择「Chat」栏，确保适配器路径是`train_llama3`，点击「加载模型」即可在Web UI中和微调模型进行对话。
![image.png](_html/chat_params.jpg)

在页面底部的对话框输入想要和模型对话的内容，点击「提交」即可发送消息。发送后模型会逐字生成回答，从回答中可以发现模型学习到了数据集中的内容，能够恰当地模仿诸葛亮的语气对话。
![image.png](_html/chat_result1.jpg)

点击「卸载模型」，点击“×”号取消适配器路径，再次点击「加载模型」，即可与微调前的原始模型聊天。
![image.png](_html/chat_uninstall.jpg)

重新向模型发送相同的内容，发现原始模型无法模仿诸葛亮的语气生成中文回答。
![image.png](_html/chat_result2.jpg)


## 6. 总结

本次教程介绍了如何使用PAI和LLaMA Factory框架，基于轻量化LoRA方法微调Llama-3模型，使其能够进行中文问答和角色扮演，同时通过验证集ROUGE分数和人工测试验证了微调的效果。在后续实践中，可以使用实际业务数据集，对模型进行微调，得到能够解决实际业务场景问题的本地领域大模型。